In [1]:
import os
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
from multiprocessing import cpu_count
import tensorflow as tf

Using TensorFlow backend.


In [2]:
nb_classes = 21
nb_cpus = cpu_count()//2
nb_gpus = 4
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3'

image_size = (299, 299)
input_shape= (299, 299, 3)

In [3]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(1.0)(p_out)
    predictions = Dense(nb_classes, activation='softmax')(p_out)
    model = Model(inputs=base_model.input, outputs=predictions)
    
model = multi_gpu_model(model, gpus=nb_gpus)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_5 (

In [4]:
model.load_weights('batch6.3_008_0.0763.hdf5')

In [5]:
batch_size = 64
test_path = "/home/ssd0/Develop/liyu/xcp-batch6.3/CELLS-new-half299/valid"

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_path, 
                                         target_size=image_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
all_test_results = model.predict_generator(test_generator, 
                                           len(test_generator), 
                                           workers=nb_cpus, 
                                           use_multiprocessing=True,
                                           verbose=1)
all_labels = test_generator.classes

Found 49086 images belonging to 21 classes.
767/767 [==============================] - 186s 242ms/step


In [6]:
class_label_dict = test_generator.class_indices
print(test_generator.class_indices)
def get_key(dict_, value):
    return [k for k, v in dict_.items() if v == value]

# create class num lens dict, every dict store current class predict num
total_predictions_dict = {}
for row_class_name, label in class_label_dict.items():
    total_predictions_dict[row_class_name] = {}
    for column_class_name, label in class_label_dict.items():
        total_predictions_dict[row_class_name][column_class_name] = 0

{'SCC_G': 17, 'ACTINO': 0, 'HSIL_B': 8, 'LSIL_E': 11, 'CC': 4, 'FUNGI': 6, 'HSIL_S': 10, 'TRI': 19, 'LSIL_F': 12, 'AGC_A': 1, 'EC': 5, 'VIRUS': 20, 'SC': 16, 'MC': 13, 'ASCUS': 3, 'HSIL_M': 9, 'PH': 14, 'AGC_B': 2, 'RC': 15, 'SCC_R': 18, 'GEC': 7}


In [16]:
thresh = 0.95

for i, label in enumerate(all_labels):
    predict_index = np.argmax(all_test_results[i])
    predict_det = all_test_results[i][predict_index]
    if (predict_det > thresh):
        # get the first result
        label_class_name = get_key(class_label_dict, label)[0]
        test_class_name = get_key(class_label_dict, np.argmax(all_test_results[i]))[0]
#     print(label_class_name)
#     print(total_predictions_dict[label_class_name].keys())
        total_predictions_dict[label_class_name][test_class_name] += 1

In [17]:
print(total_predictions_dict['RC'])

{'SCC_G': 0, 'ACTINO': 0, 'HSIL_B': 0, 'SCC_R': 0, 'CC': 0, 'FUNGI': 0, 'ASCUS': 0, 'HSIL_S': 0, 'LSIL_F': 0, 'AGC_A': 0, 'EC': 0, 'VIRUS': 0, 'SC': 0, 'TRI': 0, 'LSIL_E': 0, 'MC': 0, 'AGC_B': 0, 'GEC': 34, 'PH': 0, 'RC': 3378, 'HSIL_M': 1}


In [18]:
import csv
out = open('confusion_matrix_95.csv','a', newline='')
csv_write = csv.writer(out,dialect='excel')

# write the title
line = [class_name for class_name, label in class_label_dict.items()]
line = [" "] + line + ["TOTAL"] + ["ACC"]
csv_write.writerow(line)

# write rows

true_num = 0
all_num = 0

for row_class_name, label in class_label_dict.items():
    one_class_total_predict = 0
    line = [row_class_name]
    for column_class_name, label in class_label_dict.items():
        one_class_total_predict += total_predictions_dict[row_class_name][column_class_name]
    
    for column_class_name, label in class_label_dict.items():
        one_class_cur_predict = total_predictions_dict[row_class_name][column_class_name]
        # acc
        #acc = round((one_class_cur_predict / one_class_total_predict), 4)
        #line.append(str(acc))
        # num
        line.append(one_class_cur_predict)
    print(one_class_total_predict)
    print(str(one_class_total_predict))
    line.append(str(one_class_total_predict))
    line.append(round((total_predictions_dict[row_class_name][row_class_name] / one_class_total_predict), 4))       
    print(line)
    csv_write.writerow(line)
    
    true_num += total_predictions_dict[row_class_name][row_class_name]
    all_num += one_class_total_predict
    
csv_write.writerow(["ALL_ACC"] + [round((true_num / all_num), 4)])
    
out.close()

3531
3531
['SCC_G', 3393, 0, 0, 0, 0, 0, 119, 0, 0, 12, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 6, '3531', 0.9609]
11266
11266
['ACTINO', 0, 11266, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '11266', 1.0]
3733
3733
['HSIL_B', 0, 0, 3637, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 41, 0, 30, 12, 0, 0, '3733', 0.9743]
5028
5028
['LSIL_E', 0, 0, 0, 4967, 0, 0, 0, 0, 25, 0, 0, 0, 0, 12, 16, 0, 8, 0, 0, 0, 0, '5028', 0.9879]
15414
15414
['CC', 0, 0, 0, 0, 15407, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, '15414', 0.9995]
9089
9089
['FUNGI', 0, 0, 0, 0, 0, 9084, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, '9089', 0.9994]
32361
32361
['HSIL_S', 153, 0, 0, 0, 0, 0, 31814, 0, 0, 0, 0, 9, 353, 0, 2, 28, 0, 0, 0, 2, 0, '32361', 0.9831]
33631
33631
['TRI', 0, 0, 0, 0, 0, 0, 0, 33613, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, '33631', 0.9995]
2729
2729
['LSIL_F', 0, 0, 0, 24, 0, 0, 0, 0, 2498, 0, 0, 0, 0, 0, 195, 0, 0, 0, 12, 0, 0, '2729', 0.9154]
3049
3049
['AGC_A', 0, 0, 0, 0, 0, 0, 0, 0, 0, 30